Homeless Dataset - Import & Clean Data

In [ ]:
import pandas as pd

#import homeless dataset and clean up data
df = pd.read_csv(f'../src/data/homeless_2022.csv', sep='\t')
df = df.rename(columns={"Name": 'state'})
df = df.rename(columns={"Avg. ValueToDisplay": 'homeless #'})
df["homeless #"] = df["homeless #"].str.replace(",","").astype(int)
df = df.drop(columns=[df.columns[3]])

#rename state column to match wealth datasets
abrev_to_state_mapping = { #from: https://medium.com/@jason_the_data_scientist/python-mapping-state-abbreviations-to-state-and-vice-versa-in-pandas-e4cd24edefb0
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'PR': 'Puerto Rico',
        'VI': 'Virigin Islands'
    }
df['state'].replace(abrev_to_state_mapping,inplace=True) #todo - check why compiler complains about this line


print(df.head())

Analyze homeless dataset in relation to wealth dataset

In [ ]:
#Analysis - extract data from the corresponding year wealth dataset and analyze relevant statistics

# neccessary to import modules from src
import os
import sys
import math 
import numpy
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pathlib import Path
from src.scripts.read_parse import read_csv_data, parse_data_remove_useless_cols
from src.scripts.analyze import parse_data_count_based_on_col, display_all_graphs


#get # billionares per state
billionare_data = pd.read_csv(f'../src/data/billionaires_2022.csv')
billionare_data = parse_data_remove_useless_cols(billionare_data)
billionare_data =  billionare_data[billionare_data['state'].isin(abrev_to_state_mapping.values())]#filter out all non-US states
#print(billionare_data.head())
state_counts = parse_data_count_based_on_col(billionare_data, 'state')

#find billionare to homeless ratio
df = pd.merge(df,state_counts,on='state')
df = df.rename(columns={"#_of_people": 'billionares #'})
df['ratio'] = df['billionares #']/df['homeless #']*1000 #billionares per 1k homeless ratio
print('billionare to homeless ratio:')
print(df.head())

#compute billionare wealth per state (sum of net worth of billionares in state)
billionare_sums = billionare_data.groupby('state')['realTimeWorth'].sum()
print("\nstate billionare sums:\n")
print(billionare_sums.head())

#plot homeless to wealth in state ratio (how does billionare wealth correspond to state wealth?)
df = pd.merge(df,billionare_sums,on='state')
df = df.rename(columns={'realTimeWorth': 'state_wealth_sum'})
df['(wealth_ratio'] = df['state_wealth_sum']/df['homeless #']*1000 #billionare money (thousands $) per homeless

print(df.head())

#would be interesting to analyze whether states with higher billionare ratio also have higher homeless ratio

#plot homeless ratio per state

#plot homeless demographics?


#todo - perform estimation regarding expenses for homeless per state

In [ ]:
#Visualizations Experimentation



from src.scripts.visualizations.scatter_usa import plot_usa_map_with_data
    

#issue - numbers cut off after two digits
df = df.rename(columns={'homeless #': '#_of_people'})
#df = df.rename(columns={'ratio': '#_of_people'}) 
print(df)
plot_usa_map_with_data(df)